In [2]:
pip install wget

  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9681 sha256=0287daf25ed7de9f6fcf278ee665fd8860402ac43ac1d26bfce10185535e8de4
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
import numpy as np
from ftplib import FTP
import os, sys, os.path
import pandas as pd
from datetime import datetime

import wget
from urllib import request
from tqdm import tqdm

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


**Download CHIRPS dataset in 0.25 and 0.05 degrees resolution**

In [ ]:
#change resolution
res = '05'  #for 0.25 choose 25; for 0.05 choose 05

ddir='gdrive/MyDrive/rainfall-forecast/precipitation_data/CHIRPS/' + str(res)
ftp = FTP('data.chc.ucsb.edu')
ftp.login()


directory = '/pub/org/chg/products/CHIRPS-2.0/global_daily/netcdf/p' +str(res) + str('/')
print ('Changing to ' + directory)

ftp.cwd(directory)

filenames = ftp.nlst()

#first element in list is a monthly directory link

del filenames[0]

for filename in filenames:
  local_filename = os.path.join(ddir, filename)
  print(local_filename)
  file = open(local_filename, 'wb')
  ftp.retrbinary('RETR '+ filename, file.write)
     
  file.close()

ftp.quit()

NameError: ignored

**Download IMERG dataset**

In [ ]:
username = "robert.keppler@live.com"
password = "Counter1@"
url = "https://gpm1.gesdisc.eosdis.nasa.gov/data/GPM_L3/GPM_3IMERGDL.06/2000/06/"
filename = '3B-DAY-L.MS.MRG.3IMERG.20000601-S000000-E235959.V06.nc4'#

redirectHandler = request.HTTPRedirectHandler()
cookieProcessor = request.HTTPCookieProcessor()
passwordManager = request.HTTPPasswordMgrWithDefaultRealm()
passwordManager.add_password(None, "https://urs.earthdata.nasa.gov", username, password)
authHandler = request.HTTPBasicAuthHandler(passwordManager)
opener = request.build_opener(redirectHandler,cookieProcessor,authHandler)
request.install_opener(opener)
request.urlretrieve(url,filename)

('3B-DAY-L.MS.MRG.3IMERG.20000601-S000000-E235959.V06.nc4',
 <http.client.HTTPMessage at 0x7fd81dd0d790>)

In [ ]:
#os.chdir('gdrive/MyDrive/rainfall-forecast/precipitation_data/IMERG')

date_range = pd.date_range('06/01/2000', '04/02/2021').tolist()

filelist = []
for date in date_range:
  year = date.date().strftime('%Y')
  month = date.date().strftime('%m')
  day = date.date().strftime('%Y%m%d')
  string = year + '/' + month + '/3B-DAY-L.MS.MRG.3IMERG.' + day + '-S000000-E235959.V06.nc4'
  filelist.append(string)

dspath = 'https://gpm1.gesdisc.eosdis.nasa.gov/data/GPM_L3/GPM_3IMERGDL.06/'

for file in tqdm(filelist):
  filename = dspath + file
  wget.download(filename)

**Download CMORPH dataset**

In [ ]:
import sys, os
import requests


def check_file_status(filepath, filesize):
    sys.stdout.write('\r')
    sys.stdout.flush()
    size = int(os.stat(filepath).st_size)
    percent_complete = (size/filesize)*100
    sys.stdout.write('%.3f %s' % (percent_complete, '% Completed'))
    sys.stdout.flush()

# Try to get password
if len(sys.argv) < 2 and not 'RDAPSWD' in os.environ:
    try:
        import getpass
        input = getpass.getpass
    except:
        try:
            input = raw_input
        except:
            pass
    pswd = input('Password: ')
else:
    try:
        pswd = sys.argv[1]
    except:
        pswd = os.environ['RDAPSWD']

url = 'https://rda.ucar.edu/cgi-bin/login'
values = {'email' : 'robert.keppler@live.com', 'passwd' : 'Counter1@', 'action' : 'login'}
# Authenticate
ret = requests.post(url,data=values)
if ret.status_code != 200:
    print('Bad Authentication')
    print(ret.text)
    exit(1)
dspath = 'https://rda.ucar.edu/data/ds502.0/'

date_range = pd.date_range('12/07/2002', '08/04/2017').tolist()
date_range = pd.date_range('31/12/2014', '02/01/2015').tolist()

filelist = []
for date in date_range:
  year = date.date().strftime('%Y')
  day = date.date().strftime('%Y%m%d')
  string = 'netcdf/' + year + '/cmorph.3hr-025deg.' + day + '.nc'
  filelist.append(string)

  
for file in filelist:
    filename=dspath+file
    path = 'gdrive/MyDrive/rainfall-forecast/precipitation_data/CMORPH'
    file_base = os.path.basename(file)
    file_base = os.path.join(path, file_base)
    print(filename)
    print('Downloading',file_base)
    req = requests.get(filename, cookies = ret.cookies, allow_redirects=True, stream=True)#stream=True
    print(req.headers)
    filesize = int(req.headers['Content-length'])
    with open(file_base, 'wb') as outfile:
        chunk_size=1048576
        for chunk in req.iter_content(chunk_size=chunk_size):#chunk_size
            outfile.write(chunk)
            if chunk_size < filesize:
                check_file_status(file_base, filesize)
    check_file_status(file_base, filesize)
    print()


CMORPH daily = 0.25 degrees res

In [ ]:
import shutil
import requests
from tqdm import tqdm

path = 'https://www.ncei.noaa.gov/data/cmorph-high-resolution-global-precipitation-estimates/access/daily/0.25deg/'
date_range = pd.date_range('01/01/1998', '31/10/2020').tolist()#31/10/2020

filelist = []
for date in date_range:
  year = date.date().strftime('%Y')
  month = date.date().strftime('%m')
  day = date.date().strftime('%Y%m%d')
  string = year + '/' + month + '/CMORPH_V1.0_ADJ_0.25deg-DLY_00Z_' + day + '.nc'
  filelist.append(string)

dspath = 'https://www.ncei.noaa.gov/data/cmorph-high-resolution-global-precipitation-estimates/access/daily/0.25deg/'

for file in tqdm(filelist):
  filename = dspath + file
  path = 'gdrive/MyDrive/rainfall-forecast/precipitation_data/CMORPH'
  file_base = os.path.basename(file)
  file_base = os.path.join(path, file_base)
  req = requests.get(filename)
  if response.status_code == 200:
    with open(file_base, 'wb') as f:
      f.write(response.content)

100%|██████████| 8340/8340 [1:40:18<00:00,  1.39it/s]


CMORPH 30 min = 8km res

In [8]:
import shutil
import requests
from tqdm.auto import tqdm

path = 'https://www.ncei.noaa.gov/data/cmorph-high-resolution-global-precipitation-estimates/access/30min/8km//'
date_range = pd.date_range('11/01/2020', '31/12/2020', freq = '60min').tolist()#31/10/2020

filelist = []
for date in date_range:
  year = date.date().strftime('%Y')
  month = date.date().strftime('%m')
  daily = date.date().strftime('%d')
  day = date.date().strftime('%Y%m%d')
  hour = date.strftime('%H')
  #print(hour)
  string = year + '/' + month + '/' + daily + '/CMORPH_V1.0_ADJ_8km-30min_' + day + hour + '.nc'
  filelist.append(string)

dspath = 'https://www.ncei.noaa.gov/data/cmorph-high-resolution-global-precipitation-estimates/access/30min/8km/'

#os.chdir('gdrive/MyDrive/rainfall-forecast/precipitation_data/CMORPH/30min')

#os.getcwd()
for file in tqdm(filelist):
  filename = dspath + file
  wget.download(filename, out = 'gdrive/MyDrive/rainfall-forecast/precipitation_data/CMORPH/30min2')

CMORPH PRELIMINARY DATA 30 MIN, 8KM RES, FROM 01/11/20 TO 31/12/20

In [ ]:
import shutil
import requests
from tqdm.auto import tqdm

#path = 'https://www.ncei.noaa.gov/data/cmorph-high-resolution-global-precipitation-estimates/access/30min/8km//'
date_range = pd.date_range('11/01/2020', '31/12/2020', freq = '60min').tolist()#31/10/2020

filelist = []
for date in date_range:
  year = date.date().strftime('%Y')
  month = date.date().strftime('%m')
  daily = date.date().strftime('%d')
  day = date.date().strftime('%Y%m%d')
  hour = date.strftime('%H')
  #print(hour)
  string = year + '/' + year + month + '/' + '/CMORPH_V.0_ADJ_8km-30min_' + day + hour + '.nc'
  filelist.append(string)

dspath = 'https://www.ncei.noaa.gov/data/cmorph-high-resolution-global-precipitation-estimates/access/30min/8km/123466'

#os.chdir('gdrive/MyDrive/rainfall-forecast/precipitation_data/CMORPH/30min')

#os.getcwd()
for file in tqdm(filelist):
  filename = dspath + file
  #wget.download(filename, out = 'gdrive/MyDrive/rainfall-forecast/precipitation_data/CMORPH/30min')

In [ ]:
filelist

['2020/11/01/CMORPH_V1.0_ADJ_8km-30min_2020110100.nc',
 '2020/11/01/CMORPH_V1.0_ADJ_8km-30min_2020110101.nc',
 '2020/11/01/CMORPH_V1.0_ADJ_8km-30min_2020110102.nc',
 '2020/11/01/CMORPH_V1.0_ADJ_8km-30min_2020110103.nc',
 '2020/11/01/CMORPH_V1.0_ADJ_8km-30min_2020110104.nc',
 '2020/11/01/CMORPH_V1.0_ADJ_8km-30min_2020110105.nc',
 '2020/11/01/CMORPH_V1.0_ADJ_8km-30min_2020110106.nc',
 '2020/11/01/CMORPH_V1.0_ADJ_8km-30min_2020110107.nc',
 '2020/11/01/CMORPH_V1.0_ADJ_8km-30min_2020110108.nc',
 '2020/11/01/CMORPH_V1.0_ADJ_8km-30min_2020110109.nc',
 '2020/11/01/CMORPH_V1.0_ADJ_8km-30min_2020110110.nc',
 '2020/11/01/CMORPH_V1.0_ADJ_8km-30min_2020110111.nc',
 '2020/11/01/CMORPH_V1.0_ADJ_8km-30min_2020110112.nc',
 '2020/11/01/CMORPH_V1.0_ADJ_8km-30min_2020110113.nc',
 '2020/11/01/CMORPH_V1.0_ADJ_8km-30min_2020110114.nc',
 '2020/11/01/CMORPH_V1.0_ADJ_8km-30min_2020110115.nc',
 '2020/11/01/CMORPH_V1.0_ADJ_8km-30min_2020110116.nc',
 '2020/11/01/CMORPH_V1.0_ADJ_8km-30min_2020110117.nc',
 '2020/11/

In [ ]:
!ls gdrive/MyDrive/rainfall-forecast/precipitation_data/CMORPH/30min

In [ ]:
!ls gdrive/MyDrive/rainfall-forecast/precipitation_data/CMORPH/30min